In [9]:
import scanpy as sc 
import pandas as pd

In [8]:
# See script /nfs/team298/sm54/BoneAtlasProject/src/processing/06_generate_combined_h5ad.py

In [ ]:
adata_combined = sc.concat([adata1, linnarson], join='inner', index_unique=None)


In [1]:
# The gene names were not saved in the combined so i reran thsi step in 07_generate_combined_h5ad

In [2]:
import scanpy as sc 

In [3]:
combined_adata= sc.read_h5ad("/lustre/scratch124/cellgen/haniffa/users/sm54/data/combined_adata_bone_cross_organ_brain_for_scvi.h5ad")

In [18]:
len(combined_adata.obs['Sanger_ID'].unique())

924

In [19]:
len(metadata['Sample_ID'].unique())

930

In [28]:
metadata= pd.read_csv('/nfs/team298/sm54/BoneAtlasProject/metadata/sample_metadata/COMBINED_BONE_CROSS_ORGAN_BRAIN_METADATA_FINAL.csv', index_col=0)

In [29]:
((set(metadata['Sample_ID']) - set(combined_adata.obs["Sanger_ID"])))

{'10X109_5',
 '10X96_1',
 '10X96_2',
 'F83_TH_EPCAM',
 'FCAImmP7241243',
 'HCA_BN_F12482364'}

In [30]:
((set(combined_adata.obs["Sanger_ID"])- set(metadata['Sample_ID'])))

set()

In [ ]:
((set(combined_adata.obs["Sanger_ID"])- set(metadata['Sample_ID'])))

In [31]:
import pandas as pd

def add_metadata_to_obs(
    adata,
    df: pd.DataFrame,
    df_key: str,
    obs_key: str = None,
):
    """
    Merge EVERY column of `df` into `adata.obs` by matching
      df[df_key]  ↔  adata.obs[obs_key] (or adata.obs_names if obs_key is None).

    Keeps duplicates in df_key by dropping all but the first.
    Leaves adata.obs index untouched.
    """

    # 1. Sanity checks
    assert df_key in df.columns, f"{df_key!r} not in metadata DataFrame"
    if obs_key is not None:
        assert obs_key in adata.obs.columns, f"{obs_key!r} not in adata.obs"

    # 2. Deduplicate metadata on the key, keep only first
    df_unique = df.drop_duplicates(subset=[df_key], keep="first").copy()

    # 3. Build a lookup Series indexed by the key
    df_indexed = df_unique.set_index(df_key)

    # 4. Pick the “left” key from adata.obs
    if obs_key is None:
        # map from obs_names
        left_index = pd.Series(adata.obs_names, index=adata.obs_names, name=df_key)
    else:
        # map from an existing obs column
        left_index = adata.obs[obs_key].rename(df_key)

    # 5. For every column in df_indexed (including df_key itself), map into adata.obs
    for col in df_indexed.columns:
        adata.obs[col] = left_index.map(df_indexed[col])

    # 6. Report if anything expected went missing
    missing = adata.obs[df_indexed.columns].isna().any(axis=1).sum()
    if missing:
        print(f"⚠️  {missing} cells did not get full metadata from `{df_key}` join")

    return adata


In [32]:
 #)add metadata keyed by Sample_ID → obs['Sanger_ID']
add_metadata_to_obs(combined_adata, metadata,   df_key='Sample_ID', obs_key='Sanger_ID')


⚠️  1579208 cells did not get full metadata from `Sample_ID` join


AnnData object with n_obs × n_vars = 5786552 × 36601
    obs: 'Age', 'Cell_ID', 'Sanger_ID', 'barcode_raw', 'gene_counts_outlier', 'log1p_n_genes_by_counts', 'log1p_total_counts', 'log1p_total_counts_mt', 'log1p_total_counts_rb', 'mt_outlier', 'n_genes_by_counts', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'pct_counts_in_top_50_genes', 'pct_counts_mt', 'pct_counts_rb', 'total_counts', 'total_counts_mt', 'total_counts_outlier', 'total_counts_rb', 'FACS_Sorting', 'Method', 'Multiplexed', 'Dataset', 'Anatomical_Site', 'Organ', '10X_Chemistry', 'Donor_ID', 'Accession_ID', 'Organism', 'Sex', 'Expected_Donors', 'Study', 'Technology', 'Age(PCW)'

In [34]:
metadata

,FACS_Sorting,Age,Method,Multiplexed,Dataset,Anatomical_Site,Organ,10X_Chemistry,Donor_ID,Accession_ID,Organism,Sample_ID,Sex,Expected_Donors,Study,Technology,Age(PCW)
0,Total,6.0,3GEX,No,Linnarson,Spinal_cord_lumbar,Brain,10x3'v2,XDD:326,EGAD00001006049,Human,10X147_5,unknown,1,Braun_et_al_2023,10X_scRNA-seq,6.0
1,Total,10.0,3GEX,No,Linnarson,Forebrain,Brain,10x3'v2,XHU:297,EGAD00001006049,Human,10X101_8,unknown,1,Braun_et_al_2023,10X_scRNA-seq,10.0
2,Total,10.0,3GEX,No,Linnarson,Forebrain,Brain,10x3'v2,XHU:297,EGAD00001006049,Human,10X101_7,unknown,1,Braun_et_al_2023,10X_scRNA-seq,10.0
3,Total,10.0,3GEX,No,Linnarson,Forebrain,Brain,10x3'v2,XHU:297,EGAD00001006049,Human,10X101_6,unknown,1,Braun_et_al_2023,10X_scRNA-seq,10.0
4,Total,10.0,3GEX,No,Linnarson,Forebrain,Brain,10x3'v2,XHU:297,EGAD00001006049,Human,10X101_5,unknown,1,Braun_et_al_2023,10X_scRNA-seq,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
925,Total,"9.2,9.9,12.5",10X_multiome,Yes,Cross_Organ,multiplexed,Meninges,10x_multiome,multiplexed,Sanger,Homo sapiens,HCA_BRA_F13774552,Unknown,3,Sanger,10X_multiome,"9.2,9.9,12.5"
926,Total,"9.2,9.9,12.5",10X_multiome,Yes,Cross_Organ,multiplexed,Meninges,10x_multiome,multiplexed,Sanger,Homo sapiens,HCA_BRA_F13774553,Unknown,3,Sanger,10X_multiome,"9.2,9.9,12.5"
927,Total,"9.2,9.9,12.5",10X_multiome,Yes,Cross_Organ,sb-ant,Meninges,10x_multiome,multiplexed,Sanger,Homo sapiens,HCA_BRA_F13774554,Unknown,3,Sanger,10X_multiome,"9.2,9.9,12.5"
928,Total,"9.2,9.9,12.5",10X_multiome,Yes,Cross_Organ,sb-post,Meninges,10x_multiome,multiplexed,Sanger,Homo sapiens,HCA_BRA_F13774555,Unknown,3,Sanger,10X_multiome,"9.2,9.9,12.5"


In [36]:
print("Unique Sample_IDs in df:", metadata['Sample_ID'].nunique())
print("Unique Sample_IDs in adata.obs:", combined_adata.obs['Sanger_ID'].nunique())


Unique Sample_IDs in df: 930
Unique Sample_IDs in adata.obs: 924


In [38]:
matched_ids = set(metadata['Sample_ID']) & set(combined_adata.obs['Sanger_ID'])
print("Matching Sample_IDs:", len(matched_ids))

Matching Sample_IDs: 924


In [39]:
# Get how many adata.obs rows have NaNs for each column added
added_cols = metadata.columns.tolist()
combined_adata.obs[added_cols].isna().sum().sort_values(ascending=False)

KeyError: "['Sample_ID'] not in index"

In [40]:
added_cols = [col for col in metadata.columns if col in combined_adata.obs.columns]
combined_adata.obs[added_cols].isna().sum().sort_values(ascending=False)

Sex                1579208
FACS_Sorting             0
Age                      0
Method                   0
Multiplexed              0
Dataset                  0
Anatomical_Site          0
Organ                    0
10X_Chemistry            0
Donor_ID                 0
Accession_ID             0
Organism                 0
Expected_Donors          0
Study                    0
Technology               0
Age(PCW)                 0
dtype: int64

In [41]:
combined_adata.obs["Organ"].value_counts()

Organ
Brain                  2643336
skin                    516498
Spine                   484767
liver                   273073
Calvaria                198655
spleen                  162187
Femur                   160401
yolk sac                157300
thymus                  147308
Femur                   139069
Meninges                129196
gut                     118357
Knee                    108518
Skull base               95907
Shoulder                 95794
Hip                      80154
Skull base               46262
kidney                   34509
Forelimb                 31878
Long bone                29609
Skull base/Calvaria      28270
Tibia                    16418
yolk sac stalk           16414
Hip/Knee                 15901
Limb                     13757
Spine                    11175
Hip/Shoulder              9802
gut                       9166
Fibula                    7072
Tibia                     5799
Name: count, dtype: int64

In [42]:
combined_adata.obs["Organ"] = combined_adata.obs["Organ"].str.strip()


In [43]:
combined_adata.obs["Organ"].value_counts()

Organ
Brain                  2643336
skin                    516498
Spine                   495942
Femur                   299470
liver                   273073
Calvaria                198655
spleen                  162187
yolk sac                157300
thymus                  147308
Skull base              142169
Meninges                129196
gut                     127523
Knee                    108518
Shoulder                 95794
Hip                      80154
kidney                   34509
Forelimb                 31878
Long bone                29609
Skull base/Calvaria      28270
Tibia                    22217
yolk sac stalk           16414
Hip/Knee                 15901
Limb                     13757
Hip/Shoulder              9802
Fibula                    7072
Name: count, dtype: int64

In [44]:
combined_adata.write_h5ad("/lustre/scratch124/cellgen/haniffa/users/sm54/data/combined_adata_bone_cross_organ_brain_for_scvi.h5ad")

In [45]:
combined_adata

AnnData object with n_obs × n_vars = 5786552 × 36601
    obs: 'Age', 'Cell_ID', 'Sanger_ID', 'barcode_raw', 'gene_counts_outlier', 'log1p_n_genes_by_counts', 'log1p_total_counts', 'log1p_total_counts_mt', 'log1p_total_counts_rb', 'mt_outlier', 'n_genes_by_counts', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'pct_counts_in_top_50_genes', 'pct_counts_mt', 'pct_counts_rb', 'total_counts', 'total_counts_mt', 'total_counts_outlier', 'total_counts_rb', 'FACS_Sorting', 'Method', 'Multiplexed', 'Dataset', 'Anatomical_Site', 'Organ', '10X_Chemistry', 'Donor_ID', 'Accession_ID', 'Organism', 'Sex', 'Expected_Donors', 'Study', 'Technology', 'Age(PCW)'

In [ ]:
combined_adata.obs["Organ"].isin(['Brain','Calvaria','Skull base','Meninges','Skull base/Calvaria'])]

In [48]:
celltypist_bone_atlas= pd.read_csv('/lustre/scratch124/cellgen/haniffa/users/sm54/data/Bone_Atlas/all_samples_predictions_filtered_bone_atlas.csv')
celltypist_cross_organ= pd.read_csv('/lustre/scratch124/cellgen/haniffa/users/sm54/data/Cross_Organ/all_samples_predictions_filtered.csv')
celltypist_linnarson= pd.read_csv('/lustre/scratch124/cellgen/haniffa/users/sm54/data/Linnarson/all_samples_predictions_filtered_linnarson.csv')

In [49]:
celltypist_linnarson

,Cell_ID,Sanger_ID,predicted_labels,over_clustering,majority_voting
0,TACCTATTCCACGAAT-10X147_5,10X147_5,Medulla neuron,0,Medulla neuron
1,TACTTGTTCTAGCACA-10X147_5,10X147_5,Medulla radial glia,1,Medulla radial glia
2,AACTGGTGTGCAACTT-10X147_5,10X147_5,Medulla neuron,0,Medulla neuron
3,CTCAGAATCTGCGTAA-10X147_5,10X147_5,Medulla neuron,0,Medulla neuron
4,GAACGGACAGACAAGC-10X147_5,10X147_5,Medulla neuron,0,Medulla neuron
...,...,...,...,...,...
2643331,TGTTACTCACTCTCGT-10X187_1_ABC_1,10X187_1_ABC_1,Neurons,7,Cortex neuronal IPC
2643332,TCGTAGACAGACTCTA-10X187_1_ABC_1,10X187_1_ABC_1,Periosteal dura cells,7,Cortex neuronal IPC
2643333,TCGTAGACATGGAACG-10X187_1_ABC_1,10X187_1_ABC_1,Hippocampus glioblast,51,Cortex glioblast
2643334,CATCGTCGTGTCCACG-10X187_1_ABC_1,10X187_1_ABC_1,DC2,33,Cortex glioblast


In [52]:
2643336 + 1564008 + 1579208

5786552

In [50]:
celltypist_cross_organ

,Cell_ID,Sanger_ID,predicted_labels,over_clustering,majority_voting
0,CGTAGCGGTCATGCAT-Human_colon_16S7985393,Human_colon_16S7985393,Cycling fibroblast,0,Cycling fibroblast
1,TTCTCCTGTTTGTGTG-Human_colon_16S7985393,Human_colon_16S7985393,Endothelial cells,1,Endothelial cells
2,CTCTAATGTAGCCTAT-Human_colon_16S7985393,Human_colon_16S7985393,Myofibroblast,2,Cycling fibroblast
3,CTCAGAATCCTGCAGG-Human_colon_16S7985393,Human_colon_16S7985393,Cycling fibroblast,3,Cycling fibroblast
4,ACGGCCAAGAAACCGC-Human_colon_16S7985393,Human_colon_16S7985393,Hypothalamus neuron,5,Fibroblasts
...,...,...,...,...,...
1564003,TTTCACCCACTGGCTG-HCA_BRA_F13774556,HCA_BRA_F13774556,Neurons,29,Neurons
1564004,CTAGTCGAGAAAGCAT-HCA_BRA_F13774556,HCA_BRA_F13774556,Definitive erythrocytes,126,Forebrain neuronal IPC
1564005,GCTGATCCAGGTCCTG-HCA_BRA_F13774556,HCA_BRA_F13774556,Macrophage,47,Macrophage
1564006,CCATAATCACCGTTCC-HCA_BRA_F13774556,HCA_BRA_F13774556,Macrophage,47,Macrophage


In [51]:
celltypist_bone_atlas

,Cell_ID,Sanger_ID,predicted_labels,over_clustering,majority_voting
0,CGCAATGTCAGCAAGA-HCA_BN_F12482363,HCA_BN_F12482363,Skull erythroid cells,0.0,Skull erythroid cells
1,TAATCACCAATTGACT-HCA_BN_F12482363,HCA_BN_F12482363,Definitive erythrocytes,1.0,Skull erythroid cells
2,CAAGTAACATGACTAT-HCA_BN_F12482363,HCA_BN_F12482363,Coelomic epithelial cells,0.0,Skull erythroid cells
3,AAAGCCGCATGTTGCA-HCA_BN_F12482363,HCA_BN_F12482363,neural crest,0.0,Skull erythroid cells
4,TGGCGGTTCTCCTCAA-HCA_BN_F12482363,HCA_BN_F12482363,neural crest,2.0,Perisuture cels
...,...,...,...,...,...
1579203,ACCCACTCATCGATGT-5478STDY7850149,5478STDY7850149,PAX3+ skeletal muscle progenitors,50.0,PAX7+ skeletal muscle progenitors
1579204,ACTTACTTCCTGTACC-5478STDY7850149,5478STDY7850149,Interzone cells,64.0,POMC+ osteochondral progenitors
1579205,CTCTAATCAAGTAATG-5478STDY7850149,5478STDY7850149,Maturing chondrocytes,97.0,EPYC+ chondrocytes
1579206,AGAGCGACACATCCGG-5478STDY7850149,5478STDY7850149,POMC+ osteochondral progenitors,64.0,POMC+ osteochondral progenitors


In [53]:
# Step 1: Load and concatenate the prediction tables
celltypist_all = pd.concat([
    celltypist_bone_atlas,
    celltypist_cross_organ,
    celltypist_linnarson
], axis=0, ignore_index=True)

# Step 2: Drop duplicates on 'Cell_ID' if any
celltypist_all = celltypist_all.drop_duplicates(subset='Cell_ID', keep='first')

In [55]:
celltypist_all.to_csv('/lustre/scratch124/cellgen/haniffa/users/sm54/data/combined_adata_for_scvi_celltypist.csv')

In [82]:
# Step 4: Build lookup and map into obs
#celltypist_all.set_index('Cell_ID', inplace=True)

# Create series to map
predicted = combined_adata.obs["Cell_ID"].map(celltypist_all['predicted_labels'])
majority = combined_adata.obs['Cell_ID'].map(celltypist_all['majority_voting'])

# Assign to obs
combined_adata.obs['predicted_labels'] = predicted
combined_adata.obs['majority_voting'] = majority

# Step 5: Report missing cells
n_missing = combined_adata.obs[['predicted_labels', 'majority_voting']].isna().any(axis=1).sum()
if n_missing:
    print(f"⚠️  {n_missing} cells are missing predicted labels or majority voting.")
else:
    print("✅ All cells successfully annotated with CellTypist predictions.")


⚠️  211 cells are missing predicted labels or majority voting.


In [83]:
combined_adata.obs['predicted_labels'].fillna('Unknown', inplace=True)
combined_adata.obs['majority_voting'].fillna('Unknown', inplace=True)

/tmp/ipykernel_1469165/3169557396.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_adata.obs['predicted_labels'].fillna('Unknown', inplace=True)
/tmp/ipykernel_1469165/3169557396.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method

In [73]:
len(set(combined_adata.obs["Cell_ID"]) & set(celltypist_all.index))

5786552

In [74]:
combined_adata

AnnData object with n_obs × n_vars = 5786552 × 36601
    obs: 'Age', 'Cell_ID', 'Sanger_ID', 'barcode_raw', 'gene_counts_outlier', 'log1p_n_genes_by_counts', 'log1p_total_counts', 'log1p_total_counts_mt', 'log1p_total_counts_rb', 'mt_outlier', 'n_genes_by_counts', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'pct_counts_in_top_50_genes', 'pct_counts_mt', 'pct_counts_rb', 'total_counts', 'total_counts_mt', 'total_counts_outlier', 'total_counts_rb', 'FACS_Sorting', 'Method', 'Multiplexed', 'Dataset', 'Anatomical_Site', 'Organ', '10X_Chemistry', 'Donor_ID', 'Accession_ID', 'Organism', 'Sex', 'Expected_Donors', 'Study', 'Technology', 'Age(PCW)', 'predicted_labels', 'over_clustering', 'majority_voting'

In [84]:
annotation_levels= pd.read_csv('/nfs/team298/sm54/BoneAtlasProject/metadata/CellTypist_Hierarchy_Levels3_2_1.csv', index_col=0)

In [85]:
# assume you already have:
#   annotation_levels: a pd.DataFrame with columns ["Level1","Level2","Level3"]
#   adata: your AnnData with obs cols "predicted_labels" and "majority_voting"

# 1) make the two mappings
lvl1_map = annotation_levels.set_index("Level3")["Level1"].to_dict()
lvl2_map = annotation_levels.set_index("Level3")["Level2"].to_dict()

# 2) apply to both columns
for src in ["predicted_labels", "majority_voting"]:
    combined_adata.obs[f"{src}_Level1"] = combined_adata.obs[src].map(lvl1_map)
    combined_adata.obs[f"{src}_Level2"] = combined_adata.obs[src].map(lvl2_map)


In [86]:
# Step 5: Report missing cells
n_missing = combined_adata.obs[['predicted_labels_Level1', 'predicted_labels_Level2']].isna().any(axis=1).sum()
if n_missing:
    print(f"⚠️  {n_missing} cells are missing predicted labels or majority voting.")
else:
    print("✅ All cells successfully annotated with CellTypist predictions.")


✅ All cells successfully annotated with CellTypist predictions.


In [87]:
# Step 5: Report missing cells
n_missing = combined_adata.obs[['majority_voting_Level1', 'majority_voting_Level2']].isna().any(axis=1).sum()
if n_missing:
    print(f"⚠️  {n_missing} cells are missing predicted labels or majority voting.")
else:
    print("✅ All cells successfully annotated with CellTypist predictions.")


⚠️  211 cells are missing predicted labels or majority voting.


In [77]:
combined_adata

AnnData object with n_obs × n_vars = 5786552 × 36601
    obs: 'Age', 'Cell_ID', 'Sanger_ID', 'barcode_raw', 'gene_counts_outlier', 'log1p_n_genes_by_counts', 'log1p_total_counts', 'log1p_total_counts_mt', 'log1p_total_counts_rb', 'mt_outlier', 'n_genes_by_counts', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'pct_counts_in_top_50_genes', 'pct_counts_mt', 'pct_counts_rb', 'total_counts', 'total_counts_mt', 'total_counts_outlier', 'total_counts_rb', 'FACS_Sorting', 'Method', 'Multiplexed', 'Dataset', 'Anatomical_Site', 'Organ', '10X_Chemistry', 'Donor_ID', 'Accession_ID', 'Organism', 'Sex', 'Expected_Donors', 'Study', 'Technology', 'Age(PCW)', 'predicted_labels', 'over_clustering', 'majority_voting', 'predicted_labels_Level1', 'predicted_labels_Level2', 'majority_voting_Level1', 'majority_voting_Level2'

In [88]:
combined_adata.obs['majority_voting_Level1'].fillna('Unknown', inplace=True)
combined_adata.obs['majority_voting_Level2'].fillna('Unknown', inplace=True)

/tmp/ipykernel_1469165/2223353764.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_adata.obs['majority_voting_Level1'].fillna('Unknown', inplace=True)
/tmp/ipykernel_1469165/2223353764.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].

In [90]:
combined_adata.obs['majority_voting_Level2'].value_counts()

majority_voting_Level2
Neural            2628110
Fibroblast         609628
Mesenchymal        438506
Myeloid            365293
Lymphoid           306625
Muscle             285047
Erythroid          277976
Endothelial        248759
Epithelial         223567
Chondrocyte        216388
Stroma              66697
Haematopoietic      50502
Progenitor          30812
Liver               18829
MK                  15773
Endocrine            1771
Gonadal              1440
Gut                   603
Unknown               211
Pancreatic             15
Name: count, dtype: int64

In [93]:
combined_adata.obs["Sanger_ID"] = combined_adata.obs["Sanger_ID"].astype(str)


In [95]:
combined_adata.obs["Sanger_ID"].apply(type).value_counts()


Sanger_ID
<class 'str'>    5786552
Name: count, dtype: int64

In [106]:
combined_adata.obs["Sanger_ID"].value_counts()

Sanger_ID
SB_200532_11018318    42577
SB_200532_11018321    38420
HCA_BN_F13109713      34761
SB_200532_11018324    32132
HCA_BN_F12966461      27879
                      ...  
HCA_BN_F12482365         28
SB_200532_11254461       22
SB_200532_10297932       18
SB_200532_10297933       11
SB_200532_10863985        4
Name: count, Length: 924, dtype: int64

In [102]:
combined_adata.obs["Donor_ID"].value_counts()

Donor_ID
HDBR15948          264457
BRC2191            229173
2384               204557
BRC2061            177144
XDD:351            175410
                    ...  
2369                 3376
2370                 3362
BRC¬¨‚Ä†2432.00      3112
F19                  2577
HDBR15927            2258
Name: count, Length: 123, dtype: int64

In [105]:
# Step 4: Build lookup and map into obs
#celltypist_all.set_index('Cell_ID', inplace=True)

# Create series to map
Sanger_ID = combined_adata.obs["Cell_ID"].map(celltypist_all['Sanger_ID'])

# Assign to obs
combined_adata.obs['Sanger_ID'] = Sanger_ID

# Step 5: Report missing cells
n_missing = combined_adata.obs[['Sanger_ID']].isna().any(axis=1).sum()
if n_missing:
    print(f"⚠️  {n_missing} cells are missing predicted labels or majority voting.")
else:
    print("✅ All cells successfully annotated with Sanger ID.")


✅ All cells successfully annotated with Sanger ID.


In [103]:
celltypist_all

,Sanger_ID,predicted_labels,over_clustering,majority_voting
Cell_ID,,,,
CGCAATGTCAGCAAGA-HCA_BN_F12482363,HCA_BN_F12482363,Skull erythroid cells,0.0,Skull erythroid cells
TAATCACCAATTGACT-HCA_BN_F12482363,HCA_BN_F12482363,Definitive erythrocytes,1.0,Skull erythroid cells
CAAGTAACATGACTAT-HCA_BN_F12482363,HCA_BN_F12482363,Coelomic epithelial cells,0.0,Skull erythroid cells
AAAGCCGCATGTTGCA-HCA_BN_F12482363,HCA_BN_F12482363,neural crest,0.0,Skull erythroid cells
TGGCGGTTCTCCTCAA-HCA_BN_F12482363,HCA_BN_F12482363,neural crest,2.0,Perisuture cels
...,...,...,...,...
TGTTACTCACTCTCGT-10X187_1_ABC_1,10X187_1_ABC_1,Neurons,7.0,Cortex neuronal IPC
TCGTAGACAGACTCTA-10X187_1_ABC_1,10X187_1_ABC_1,Periosteal dura cells,7.0,Cortex neuronal IPC
TCGTAGACATGGAACG-10X187_1_ABC_1,10X187_1_ABC_1,Hippocampus glioblast,51.0,Cortex glioblast


In [ ]:
combined_adata.obs['majority_voting_Level1'].fillna('Unknown', inplace=True)
combined_adata.obs['majority_voting_Level2'].fillna('Unknown', inplace=True)

In [107]:
combined_adata.write_h5ad("/lustre/scratch124/cellgen/haniffa/users/sm54/data/combined_adata_bone_cross_organ_brain_for_scvi.h5ad")